# Python libraries

In [14]:
import ee
import geemap
import os
import numpy as np
import matplotlib.pyplot as plt

# Display map
To display the map the Map of geemap is used https://github.com/giswqs/geemap, which inherits from ipyleaft.Map https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html.

In [15]:
#ee.Authenticate()

In [16]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# Load Landsat 8 and Sentinel 2 images
To load the images of Landsat 8 and Sentinel 2, both images are selected from the Google Earth Engine Data Catalog and filtered to be on the same spot https://developers.google.com/earth-engine/datasets and displayed on the interactive map.

In [17]:
lon=-75.38;
lat=4.89;

#point = ee.Geometry.Point(-3.83502, 40.40595);
point = ee.Geometry.Point(lon, lat);
start = ee.Date('2019-07-01');
finish = ee.Date('2019-07-31');

# Filter a image from the Sentinel 2 surface reflectance collection, using bounds, date, and selecting the image with the least cloud cover
img_sentinel = ee.ImageCollection("COPERNICUS/S2_SR") \
            .filterBounds(point) \
            .filterDate(start, finish) \
            .sort('CLOUD_COVER', True) \
            .first()

# Select the corresponding bands for the multiespectral image
img_sentinel_multi = img_sentinel.select(['B4', 'B3', 'B2'])

# Filter a image from the Landsat 8 surface reflectance collection, using the same bounds, date, and selecting the image with the least cloud cover
img_landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
            .filterBounds(point) \
            .filterDate(start, finish) \
            .sort('CLOUD_COVER', True) \
            .first()

# Select the corresponding bands for the multiespectral image
img_landsat_multi = img_landsat.select(['B4', 'B3', 'B2'])

# Display the images on the map
Map.addLayer(img_sentinel_multi)
Map.addLayer(img_landsat_multi)

# Center the map
Map.setCenter(lon, lat, 8)

# A. Select a region of interest (roi) using the map tools
Using the tools row placed on the left of the map, draw a rectangle inside of the images, make sure it is inside of the Sentinel 2 image bounds. This rectangle will be the region of interest.

In [18]:
# feature to extract
feature = Map.draw_last_feature
roi = feature.geometry()

print('Polygon coordinates >>', roi.getInfo().get('coordinates'))

AttributeError: 'NoneType' object has no attribute 'geometry'

# B. Select a roi using the coordinates
Instead of drawing the rectangle, you can select an area by drawing a polygon using coordinates acording to the GeoJson convention https://tools.ietf.org/html/rfc7946#section-3.1.6. There is a polygon drawn by default.

In [22]:
roi = ee.Geometry.Polygon([
  [[-4.097401, 40.388201], [-4.097401, 40.572045], [-3.465607, 40.572045], [-3.465607, 40.388201], [-4.097401, 40.388201]]
]);
Map.addLayer(roi, {}, 'feature');

print('Polygon coordinates >>', roi.getInfo().get('coordinates'))

delta=0.9;

P1=[lon-delta, lat-delta];
P2=[lon+delta, lat+delta];


# feature to extract
roi = ee.Geometry.Rectangle([P1, P2]);

# add the polygon to the map
Map.addLayer(roi, {}, 'feature');
print('Polygon coordinates >>', roi.getInfo().get('coordinates'))

Polygon coordinates >> [[[-4.097401, 40.388201], [-3.465607, 40.388201], [-3.465607, 40.572045], [-4.097401, 40.572045], [-4.097401, 40.388201]]]
Polygon coordinates >> [[[-76.28, 3.9899999999999998], [-74.47999999999999, 3.9899999999999998], [-74.47999999999999, 5.79], [-76.28, 5.79], [-76.28, 3.9899999999999998]]]


# Download the roi as a TIFF file
To fuse the Landsat 8 and Sentinel 2 images, you must download the files using the second option "A TIFF file for each band", then reescale the Landsat 8 using the **Resizing_Module**. Resizing the TIFF file using Google Earth Engine, leads up to errors due to the datatype. Thus, downloading the image resized is not recommended.

The next block generates the path to store the TIFF.

In [23]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename_sentinel_xs = os.path.join(out_dir, 'sentinel_multi_sr.tif')
filename_landsat_xs = os.path.join(out_dir, 'landsat_multi_sr.tif')

### Download as raw images without resizing

* A TIFF file including all bands

In [24]:
geemap.ee_export_image(img_sentinel_multi, filename=filename_sentinel_xs, region=roi, file_per_band=False)
geemap.ee_export_image(img_landsat_multi, filename=filename_landsat_xs, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /home/thaednevol/Downloads/sentinel_multi_sr.tif
Generating URL ...
Please wait ...
Data downloaded to /home/thaednevol/Downloads/landsat_multi_sr.tif


* A TIFF file for each band

In [ ]:
geemap.ee_export_image(img_sentinel_multi, filename=filename_sentinel_xs, region=roi, file_per_band=True)
geemap.ee_export_image(img_landsat_multi, filename=filename_landsat_xs, region=roi, file_per_band=True)

Generating URL ...
Please wait ...
[WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\Edwin Hurtado\\Downloads\\sentinel_multi_sr.zip'
Generating URL ...
